### Principle Component Analysis 

#### Introduction:
PCA was invented in 1901 by Karl Pearson as an analogue of the principal axis theorem in mechanics; it was later independently developed and named by Harold Hotelling in the 1930s. Depending on the field of application, it is also named the discrete Karhunen–Loève transform (KLT) in signal processing, the Hotelling transform in multivariate quality control, proper orthogonal decomposition (POD) in mechanical engineering, singular value decomposition (SVD) of X (Golub and Van Loan, 1983), eigenvalue decomposition (EVD) of $\textbf{X}^{T}\textbf{X}$ in linear algebra, factor analysis (for a discussion of the differences between PCA and factor analysis see Ch. 7 of Jolliffe's Principal Component Analysis), Eckart–Young theorem (Harman, 1960), or empirical orthogonal functions (EOF) in meteorological science, empirical eigenfunction decomposition (Sirovich, 1987), empirical component analysis (Lorenz, 1956), quasiharmonic modes (Brooks et al., 1988), spectral decomposition in noise and vibration, and empirical modal analysis in structural dynamics. For more details refer https://en.wikipedia.org/wiki/Principal_component_analysis

A simple example has been used to demonstrate PCA. Dataset used here is in the shape of 'X'. PCA was done by eigenvalue decomposition of a data covariance (or correlation) matrix or singular value decomposition of a data matrix, after a normalization step of the initial data.

In [ ]:
# Import Required Libraries
import numpy
from matplotlib import pyplot as plt
# Plot the points
Points = numpy.array([[6, 6], [4, 4], [4, 6], [6, 4], [7, 7], [3, 3], [3, 7], [7, 3]])
plt.plot(Points[:, 0], Points[:, 1], 'rd')
plt.show()

#### Step 1
While doing PCA on any kind of dataset, the first step would be to normalize the data i.e. data should be centered around origin. Hence it will have zero mean. 

In [ ]:
# Scaling the dataset to have zero mean
Mean = numpy.mean(Points, axis=0)
Points_Norm = Points - Mean
plt.plot(Points_Norm[:, 0], Points_Norm[:, 1], 'rd')
plt.show()

#### Step 2
Calculate the Sample Covariance Matrix of the normalised data and then calculate the eigen values and eigen vectors for the Sample Covariance Matrix.
Brute force way of calculating the sample covarince matrix can be very tough when vector is very large. Easy way to do is to follow the matrix calculations. This has been taken from the following: https://en.wikipedia.org/wiki/Sample_mean_and_covariance.

\begin{align*}
    Q = \frac{1}{N-1} (\textbf{M} - \textbf{1}_{\textbf{N}}\overline{\textbf{x}}^{\textbf{T}})^{T} (\textbf{M} - \textbf{1}_{\textbf{N}}\overline{\textbf{x}}^{\textbf{T}})
\end{align*}
Q: Sample Covariance Matrix

N: Total No. of Observations

M: Column Vector of all observations = $[x_{1} x_{2} x_{3}............. x_{N}]^{T}$. This is a NxK matrix whose column j is the vector of N observations on variable j.
 
$\overline{\textbf{x}}$:  is a 1×K row vector, which is mean of each variable on all observations

$x_{i}$: is a K dimensional vector, where K is total no. of variables in each vector

$\textbf{1}_{\textbf{N}}$: is an Nx1 vector of ones

Note: Each observation $x_{i}$ is a K dimensional vector ($x_{i} \in \mathbb{R}^{K}$). We will use PCA to reduce the dimension say to D i.e. $x_{reduced} \in \mathbb{R}^{D}$ $D<<K$.

In [ ]:
row, col = Points_Norm.shape
Sample_Cov_Matrix = numpy.matmul(Points_Norm.T, Points_Norm)/(row - 1)
Eigen_Values, Eigen_Vectors = numpy.linalg.eig(Sample_Cov_Matrix)

#### Step 3
Next arrange all the eigen values in descending order and re-arrange the eigen vector matrix such that first eigen vector (First Principal Component) corresponds to the largest eigen value, second eigen vector (Second Principal Component) corresponds to the next largest eigen value and so on. Let's call this matrix as $U$. We will use the first $D$ columns of the $U$ matrix and call it $U_{reduced}$, this will be the new basis for our system.

We can project observation $x_{i}$ into the eigen space/principal space defined by $U_{reduced}$ by computing

\begin{align*}
    x_{reduced} = U_{reduced}^{T}x_{i}
\end{align*}

this transforms our original point $x_{i}$ in $\mathbb{R}^{K}$ into a point $x_{reduced}$ in some reduced data space $\mathbb{R}^{D}$.

If we want to do PCA for all points in single step then,

\begin{align*}
    \textbf{X}_{reduced} = U_{reduced}^{T}\textbf{X}^{T}
\end{align*}

where $\textbf{X}$ is NxK matrix, $\textbf{X}_{reduced}$ is a DxN matrix, $U_{reduced}$ is KxD matrix, and $U$ is KxK matrix.

In [ ]:
K = col
D = 2 # Since x is 2D vector i.e. K = 2, D can be either 1 or 2
Sort_Index = numpy.argsort(Eigen_Values)[::-1]
U = Eigen_Vectors[:, Sort_Index]
U_reduced = U[:, 0:D]
PC_Proj = numpy.matmul(U_reduced.T, Points_Norm.T)

#### Step 4
We can re-construct/inverse transform the $x_{reduced}$ in eigen space/principal space to $x_{i}$ in original space. We lose some information when we do PCA and hence when we reconstruct we will not get the original vector but something close. We lose data in the compression, but even for fairly small values of $D$, we retain the “essence” of the data. To return to the original data space, we compute:

\begin{align*}
    x^{'} = U_{reduced}x_{reduced}
\end{align*}

$x^{'}$ is close to $x_{i}$. If we want to do inverse transform for all points in single step then,

\begin{align*}
    \textbf{X}^{'} = \textbf{X}_{reduced}^{T}U_{reduced}^{T}
\end{align*}

where $\textbf{X}^{'}$ is NxK matrix, $\textbf{X}_{reduced}$ is DxN matrix and $U_{reduced}$ is KxD matrix.

In [ ]:
Points_Prime = numpy.matmul(PC_Proj.T, U_reduced.T)
plt.plot(Points_Prime[:, 0], Points_Prime[:, 1], 'gd')
plt.plot(Points_Norm[:, 0], Points_Norm[:, 1], 'rd')
plt.quiver([0], [0], [3*U[0, 1]], [0], scale_units = 'xy', scale = 1)
plt.quiver([0], [0], [0], [3*U[1, 0]], scale_units = 'xy', scale = 1)
plt.axis([-4, 4, -4, 4])
plt.show()

In [ ]:
# Plot showing the cumulative variance.
Significance = [numpy.abs(i)/numpy.sum(Eigen_Values) for i in Eigen_Values]
x = numpy.arange(1, K+1)
y = numpy.cumsum(Significance)
plt.plot(x, y)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance (%)')
plt.show()

In [ ]:
# Scree Plot showing Variance of each Principal Component
plt.plot(x, Significance)
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') # for each component
plt.show()